In [1]:
import google.cloud.bigquery as bq
import google.cloud.bigquery.dbapi as bqapi
import pandas as pd
import tqdm


tqdm.tqdm.pandas()

In [2]:
from contextlib import closing

bq_client = bq.Client()
query = "SELECT published, body FROM Articles.CleanedArticles"
results = []

with closing(bqapi.Connection(client=bq_client)) as conn:
    with closing(conn.cursor()) as cursor:
        cursor.execute(query)
        for r in cursor.fetchall():
            results.append(list(r))

result_df = pd.DataFrame(results, columns=["published", "body"])
result_df.head()

,published,body
0,2023-10-23 21:53:05+00:00,Oct 23 - The West Antarctic Ice Sheet will con...
1,2023-10-23 12:32:25+00:00,"LONDON, Oct 23 (Reuters) - Companies including..."
2,2023-10-26 01:32:04+00:00,October 26 - Top-seeded Russian Daniil Medvede...
3,2023-10-18 15:24:00.111000+00:00,"October 18 - In 2014, The Guardian published a..."
4,2023-10-18 22:22:58+00:00,Oct 18 (Reuters) - A U.S. appeals court upheld...


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
docs = nlp.pipe(result_df.body)
words_count = []
for d in tqdm.tqdm(docs, total=result_df.shape[0]):
    words_count.append(len([token for token in d]))
    
result_df["words_count"] = words_count

 92%|█████████▏| 37633/41095 [39:48<04:34, 12.60it/s]